In [1]:
import pandas as pd
import glob
import json

Concatenate all data into one DataFrame

In [2]:
# get data file names
path =r'../Files/Data/'
filenames = glob.glob(path + "*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

In [3]:
big_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120746 entries, 0 to 120745
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   trending_date  120746 non-null  object
 1   category_id    120746 non-null  int64 
 2   publish_time   120746 non-null  object
 3   tags           120746 non-null  object
 4   views          120746 non-null  int64 
 5   likes          120746 non-null  int64 
 6   dislikes       120746 non-null  int64 
 7   comment_count  120746 non-null  int64 
 8   Country        120746 non-null  object
dtypes: int64(5), object(4)
memory usage: 8.3+ MB


In [4]:
big_frame.head()

,trending_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,Country
0,17.14.11,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,CA
1,17.14.11,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,CA
2,17.14.11,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,CA
3,17.14.11,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,CA
4,17.14.11,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,CA


Clean tags column and convert it to list of tags

In [5]:
big_frame["tags"] = big_frame["tags"].str.replace('"', '').replace("'", '')

In [6]:
big_frame["tags"] = big_frame["tags"].str.split("|")

Change category_id into human-readable category string

In [7]:
with open("../Files/Category/US_category_id.json") as f:
    category = json.load(f)

# ... then create a dictionary that maps category_id
# into a human-readable category ...
category_id = {item['id']: item['snippet']['title']
               for item in category['items']}

# ... and show the resulting dictionary.
category_id

{'1': 'Film & Animation',
 '2': 'Autos & Vehicles',
 '10': 'Music',
 '15': 'Pets & Animals',
 '17': 'Sports',
 '18': 'Short Movies',
 '19': 'Travel & Events',
 '20': 'Gaming',
 '21': 'Videoblogging',
 '22': 'People & Blogs',
 '23': 'Comedy',
 '24': 'Entertainment',
 '25': 'News & Politics',
 '26': 'Howto & Style',
 '27': 'Education',
 '28': 'Science & Technology',
 '29': 'Nonprofits & Activism',
 '30': 'Movies',
 '31': 'Anime/Animation',
 '32': 'Action/Adventure',
 '33': 'Classics',
 '34': 'Comedy',
 '35': 'Documentary',
 '36': 'Drama',
 '37': 'Family',
 '38': 'Foreign',
 '39': 'Horror',
 '40': 'Sci-Fi/Fantasy',
 '41': 'Thriller',
 '42': 'Shorts',
 '43': 'Shows',
 '44': 'Trailers'}

In [8]:
# Change category_id into human-readable category string
# using the dictionary created in previous cell
big_frame['category'] = big_frame['category_id'].astype(str).replace(category_id)
big_frame.head()

,trending_date,category_id,publish_time,tags,views,likes,dislikes,comment_count,Country,category
0,17.14.11,10,2017-11-10T17:00:03.000Z,"[Eminem, Walk, On, Water, Aftermath/Shady/Inte...",17158579,787425,43420,125882,CA,Music
1,17.14.11,23,2017-11-13T17:00:00.000Z,"[plush, bad unboxing, unboxing, fan mail, idub...",1014651,127794,1688,13030,CA,Comedy
2,17.14.11,23,2017-11-12T19:05:24.000Z,"[racist superman, rudy, mancuso, king, bach, r...",3191434,146035,5339,8181,CA,Comedy
3,17.14.11,24,2017-11-12T18:01:41.000Z,"[ryan, higa, higatv, nigahiga, i dare you, idy...",2095828,132239,1989,17518,CA,Entertainment
4,17.14.11,10,2017-11-09T11:04:14.000Z,"[edsheeran, ed sheeran, acoustic, live, cover,...",33523622,1634130,21082,85067,CA,Music


Convert trending_date to datetime

In [9]:
big_frame.trending_date = pd.to_datetime(big_frame.trending_date, format='%y.%d.%m')

Convert Date to Year, Month, Day

In [10]:
date_col =big_frame['trending_date'].dt

big_frame["Year"] = date_col.year
big_frame["Month"] = date_col.month
big_frame["Day"] = date_col.day
big_frame['Weekday'] = date_col.day_name()
# big_frame['trending_date'] = date_col.strftime('%Y-%m')

In [11]:
big_frame['publish_time'] = pd.to_datetime(big_frame['publish_time'], errors='coerce')

In [12]:
# big_frame.drop(columns=['video_id','title', 'channel_title', 'category_id', 'thumbnail_link', 'comments_disabled', 'ratings_disabled',
#        'video_error_or_removed', 'description',], inplace=True)

In [13]:
big_frame.to_csv('../Files/data.csv', index=False)